In [1]:
import pandas as pd

DATA_IN_PATH='./data_in/'
TRAIN_CLEAN_DATA='train_clean.csv'
train_data = pd.read_csv(DATA_IN_PATH+TRAIN_CLEAN_DATA,header=0,quoting=3)
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.0,analyzer="char",sublinear_tf=True,ngram_range=(1,3),max_features=5000)
X=vectorizer.fit_transform(reviews)

In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
RANDON_SEED=42
TEST_SPLIT=0.2
y = np.array(sentiments)
X_train,X_eval, y_train,y_eval = train_test_split(X,y,test_size=TEST_SPLIT,random_state=RANDON_SEED)

# Modeling

In [4]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train,y_train)

LogisticRegression(class_weight='balanced')

# Evaluation

In [5]:
print("Accuracy : {}".format(lgs.score(X_eval,y_eval)))

Accuracy : 0.8598


In [6]:
TEST_CLEAN_DATA='test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH+TEST_CLEAN_DATA,header=0)

In [7]:
test_data['id']=test_data['id'].apply(lambda x : x.replace('\"',''))
test_data.head()                                    

,review,id
0,naturally film main themes mortality nostalgia...,12311_10
1,movie disaster within disaster film full great...,8348_2
2,movie kids saw tonight child loved one point k...,5828_4
3,afraid dark left impression several different ...,7186_2
4,accurate depiction small time mob life filmed ...,12128_7


In [8]:
testDataVecs = vectorizer.transform(test_data['review'])

## Prediction

In [9]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [10]:
import os

DATA_OUT_PATH='./data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids = list(test_data['id'])
answer_data=pd.DataFrame({'id':ids,'sentiment':test_predicted})

In [11]:
answer_data.to_csv(DATA_OUT_PATH+'lgs_tfidf_answer.csv',index=False)